# Running External Commands with `subprocess.run`

- DevOps automation often requires invoking existing CLI tools or scripts to leverage their functionality without re-implementing it in Python.  
- The `subprocess` module provides a secure and flexible interface to spawn child processes, control their input/output streams, and inspect their exit statuses.  
- The modern recommended method is `subprocess.run()`, which combines execution, output capture, and error handling in a single call.  

In [4]:
import subprocess 
import sys 

result = subprocess.run([sys.executable, "-c", "print('Hello from subprocess.')"], 
                        capture_output=True, 
                        text=True
)

print(f"Return code: {result.returncode}")
print(f"Stdout: {result.stdout.strip()}")

Return code: 0
Stdout: Hello from subprocess.


## Why `subprocess`? The Old Ways

- Older approaches like `os.system()` invoke a shell directly, making them vulnerable to injection and offering limited control over I/O streams.  
- The `subprocess` module was introduced to provide finer control, better security, and a consistent API across platforms.  
- Functions such as `subprocess.call()`, `check_output()`, and `Popen` exist, but `subprocess.run()` (Python 3.5+) simplifies most common use cases into one interface.  

## The subprocess.run() Function

- `args` should be a list of strings where the first element is the command and the rest are its parameters.  
- `capture_output=True` captures both `stdout` and `stderr` into the returned `CompletedProcess`.  
- `text=True` decodes bytes into strings using the system’s default encoding.  
- `check=True` raises a `CalledProcessError` for non-zero exit codes, allowing you to handle failures via exceptions.  
- `shell=False` (the default) avoids invoking a shell, preventing injection vulnerabilities; use `shell=True` only if you fully control the command string.  
- The returned `CompletedProcess` has attributes `args`, `returncode`, `stdout`, and `stderr` for introspection.  

In [7]:
import subprocess
import sys

cmd = [
    sys.executable,
    "-c",
    """print('Hello from subprocess.')
invalid_function()"""    
]

result = subprocess.run(cmd, capture_output=True, text=True)
print(type(result))
print(f"Args: {result.args}")
print(f"Stdout: {result.stdout.strip()}")
print(f"Stderr: {result.stderr.strip()}")
print(f"Return code: {result.returncode}")

<class 'subprocess.CompletedProcess'>
Args: ['C:\\Users\\Shubhesh Swain\\anaconda3\\python.exe', '-c', "print('Hello from subprocess.')\ninvalid_function()"]
Stdout: Hello from subprocess.
Stderr: Traceback (most recent call last):
  File "<string>", line 2, in <module>
    invalid_function()
    ^^^^^^^^^^^^^^^^
NameError: name 'invalid_function' is not defined
Return code: 1


## Basic Command Execution

- Construct your command as a list, choosing the tool and its arguments explicitly.  
- Use `capture_output=True` and `text=True` to get human-readable strings.  
- Inspect `result.returncode` to determine if the command succeeded (zero) or failed (non-zero).  

In [ ]:
import subprocess
import platform

if platform.system() == "Windows":
    cmd = ["ver"]
else:
    cmd = ["uname", "-a"]

result = subprocess.run(cmd, shell=False, capture_output=True, text=True)  # Using shell=True for compatibility with Windows if set to False then command will fail.
print(f"Stdout: {result.stdout.strip()}")  

Stdout: Microsoft Windows [Version 10.0.26100.6899]


## Common Pitfalls & How to Avoid Them

- Forgetting `capture_output=True` means `result.stdout` and `result.stderr` will be `None`, so you cannot inspect them.  
- Omitting `text=True` leaves you with raw bytes that require manual decoding.  
- Using `check=False` without checking `result.returncode` can let failures go unnoticed.  
- Invoking a shell with `shell=True` and untrusted input enables injection attacks—always prefer `shell=False`.  